In [1]:
import pandas as pd
import numpy as np

In [2]:
basic = pd.read_pickle('../data/pkl/raw_games_5yrs.pkl')
advanced = pd.read_pickle('../data/pkl/boxscores_advanced_team_part2.pkl')

In [3]:
basic = basic.sort_values(by=['GAME_DATE', 'GAME_ID'], ascending=False).reset_index(drop=True)

In [4]:
basic.columns

In [5]:
games_df = basic[['TEAM_ID', 'TEAM_ABBREVIATION', 'GAME_ID', 'GAME_DATE', 'HOME_TEAM', 'PTS', 'PLUS_MINUS']].copy()
games_df.head()

In [6]:
advanced.head()

In [7]:
advanced.columns

In [8]:
columns_to_drop = ['TEAM_CITY', 'MIN', 'E_OFF_RATING', 'E_DEF_RATING', 
                   'E_NET_RATING', 'AST_RATIO', 'E_TM_TOV_PCT', 'USG_PCT', 
                   'E_USG_PCT', 'E_PACE', 'PACE_PER40', 'PIE']

In [9]:
advanced = advanced.drop(columns=columns_to_drop)
advanced

In [10]:
advanced['GAME_ID'] = advanced['GAME_ID'].astype('int32')

In [11]:
advanced = advanced.merge(games_df.drop(columns=['TEAM_ID']), on=['GAME_ID', 'TEAM_ABBREVIATION'])

In [12]:
advanced.head()

Reorder the data frame

#### DROP ROWS THAT DONT HAVE A MATCHING GAME ID

In [13]:
value_counts = advanced['GAME_ID'].value_counts()

In [14]:
unique_values = value_counts[value_counts == 1].index.tolist()

In [15]:
advanced = advanced[~advanced['GAME_ID'].isin(unique_values)].reset_index(drop=True)

In [16]:
advanced_desc = advanced.sort_values(by=['GAME_DATE'], ascending=True).copy()

In [17]:
advanced_desc

#### Data Frame is ready to be preprocessed

In [18]:
non_eng_features = ['TEAM_ABBREVIATION', 'GAME_ID', 'TEAM_ID', 'TEAM_NAME', 
                        'GAME_DATE', 'HOME_TEAM', 'PTS', 'PLUS_MINUS']

In [19]:
eng_features = advanced_desc.drop(columns=non_eng_features).columns.tolist()

In [20]:
eng_features

In [21]:
def roll(df, roll_number = 10, procedure = '', suff = '_Roll', selected_columns=[]):
    
    df_rolling = df[selected_columns + ["TEAM_ABBREVIATION"]]
    df_rolling = df_rolling.groupby(["TEAM_ABBREVIATION"], group_keys=False)
    
    def find_team_averages(team):
        return team.rolling(roll_number).mean()

    def find_team_medians(team):
        return team.rolling(roll_number).median()
    
    def find_team_stds(team):
        return team.rolling(roll_number).std()
    
    def find_team_wma(team):
        return team.rolling(roll_number).std()

    if procedure == 'median':
        df_rolling = df_rolling.apply(find_team_medians)
    elif procedure == 'std':
        df_rolling = df_rolling.apply(find_team_stds)    
    else:
        procedure = 'mean'
        df_rolling = df_rolling.apply(find_team_averages)
        
    df_rolling = df_rolling[selected_columns]
    df_rolling = df_rolling.sort_index()
    
    new_column_names = {}
    for col in df_rolling.columns:
        new_column_names[col] = col + suff + '_' + procedure

    df_rolling = df_rolling.rename(columns=new_column_names)
    return df_rolling

In [22]:
df_test = roll(df = advanced_desc, roll_number=5, procedure='mean', selected_columns=eng_features)
df_test2 = roll(df = advanced_desc, roll_number=5, procedure='median', selected_columns=eng_features)
df_test3 = roll(df = advanced_desc, roll_number=5, procedure='std', selected_columns=eng_features)

In [23]:
advanced = advanced.merge(df_test, left_index=True, right_index=True)
advanced = advanced.merge(df_test2, left_index=True, right_index=True)
advanced = advanced.merge(df_test3, left_index=True, right_index=True)

In [24]:
#advanced_desc = advanced_desc.groupby('TEAM_ABBREVIATION', as_index=False, group_keys=False)[eng_features].rolling(5).mean()
#advanced_desc.drop(columns=['TEAM_ABBREVIATION']).sort_index()
#advanced[eng_features] = advanced_desc.drop(columns=['TEAM_ABBREVIATION']).sort_index()

In [25]:
advanced.columns

In [26]:
advanced = advanced.sort_values(by=['GAME_DATE', 'GAME_ID', 'HOME_TEAM'], ascending=False).reset_index(drop=True)

In [27]:
adv_home = advanced.iloc[::2].copy()
adv_away = advanced.iloc[1::2].copy()

In [28]:
columns_away = {}
columns_home = {}
columns_to_merge = []
for column in advanced.columns:
    if column == 'GAME_ID' or column == 'PLUS_MINUS' or column == 'GAME_DATE':
        continue
    columns_to_merge.append(column + '_a')
    columns_away[column] = column + '_a'
    columns_home[column] = column + '_h'

In [29]:
columns_away

In [30]:
adv_away.rename(columns=columns_away, inplace=True)
adv_home.rename(columns=columns_home, inplace=True)

In [31]:
columns_to_merge.append('GAME_ID')

In [32]:
merged_df = adv_home.merge(adv_away[columns_to_merge], on=['GAME_ID'])

In [33]:
merged_df = merged_df.dropna()

#### Now the data is ready for scaling and one hot encoding

In [34]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

In [35]:
X_features_num = [col for col in merged_df.columns if 'GAME_ID' not in col 
                     and 'GAME_DATE' not in col 
                     and 'TEAM_ID' not in col 
                     and 'TEAM_NAME' not in col
                     and 'TEAM_ABBREVIATION' not in col
                     and 'PLUS_MINUS' not in col
                     and 'HOME_TEAM' not in col]

In [36]:
X_features_cat = ['TEAM_ABBREVIATION_h', 'TEAM_ABBREVIATION_a']

In [37]:
preproc_data = merged_df.copy()

In [38]:
scaler = MinMaxScaler()
preproc_data[X_features_num] = scaler.fit_transform(preproc_data[X_features_num])

In [39]:
ohe = OneHotEncoder(sparse=False)
ohe.fit(preproc_data[X_features_cat])
cols = [str(team) +'_h' for team in ohe.categories_[0]] + [str(team) +'_a' for team in ohe.categories_[1]]

In [40]:
preproc_data[cols]=ohe.transform(preproc_data[X_features_cat])

In [41]:
X_features = [col for col in preproc_data.columns if 'GAME_ID' not in col 
                     and 'GAME_DATE' not in col 
                     and 'TEAM_ID' not in col 
                     and 'TEAM_NAME' not in col
                     and 'TEAM_ABBREVIATION' not in col
                     and 'PLUS_MINUS' not in col
                     and 'HOME_TEAM' not in col
                     and 'PTS' not in col]

In [42]:
len(X_features)

In [43]:
preproc_data

In [44]:
X_preproc = preproc_data[X_features]
y = preproc_data['PLUS_MINUS']

In [45]:
X_preproc.columns[1:70]

In [46]:
X_preproc.to_pickle('X_preproc_rolling_advanced_part2.pkl')

In [47]:
y.to_pickle('y_preproc_rolling_advanced_part2.pkl')

In [52]:
X_preproc_part1 = pd.read_pickle('X_preproc_rolling_advanced_part1.pkl')

In [51]:
X_preproc_part1